<div>
<b><br><font face = "Gill Sans" size = "4"><center>Heart attack prediction with logistic regression</center></font></b>
</div>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,091 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,377 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packa

<span style="font-family:Gill Sans">
<h1>Hearth attach prediction with logisic regression</h1></span>

Let us use PySpark to predict if a person can suffer a heart attack based on some medical information.

Download the dataset from [Kaggle](https://www.kaggle.com/datasets/dileep070/heart-disease-prediction-using-logistic-regression?resource=download)

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, FloatType, IntegerType

# Create Spark Session in localhost
spark = SparkSession.builder.master("local").\
    appName("ITESO-LogisticRegression").\
    getOrCreate()

# Set spark context
sc = spark.sparkContext
sc.setLogLevel("ERROR")

heart_prediction_schema = StructType([
    StructField("male", IntegerType(), True),
    StructField("age", IntegerType(), True),
    StructField("education", IntegerType(), True),
    StructField("currentSmoker", IntegerType(), True),
    StructField("cigsPerDay", IntegerType(), True),
    StructField("BPMeds", IntegerType(), True),
    StructField("prevalentStroke", IntegerType(), True),
    StructField("prevalentHyp", IntegerType(), True),
    StructField("diabetes", IntegerType(), True),
    StructField("totChol", FloatType(), True),
    StructField("sysBP", FloatType(), True),
    StructField("diaBP", FloatType(), True),
    StructField("BMI", FloatType(), True),
    StructField("heartRate", IntegerType(), True),
    StructField("glucose", IntegerType(), True),
    StructField("TenYearCHD", IntegerType(), True)
])

heart_df = spark.read.format("csv").\
    option("header", "true").\
    option("mode", "permissive").\
    option("path", "/content/framingham.csv").\
    schema(heart_prediction_schema).\
    load()
heart_df.printSchema()
heart_df.show(n=10, truncate=False)

root
 |-- male: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- education: integer (nullable = true)
 |-- currentSmoker: integer (nullable = true)
 |-- cigsPerDay: integer (nullable = true)
 |-- BPMeds: integer (nullable = true)
 |-- prevalentStroke: integer (nullable = true)
 |-- prevalentHyp: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- totChol: float (nullable = true)
 |-- sysBP: float (nullable = true)
 |-- diaBP: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- heartRate: integer (nullable = true)
 |-- glucose: integer (nullable = true)
 |-- TenYearCHD: integer (nullable = true)

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI  |heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+----

In [5]:
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
heart_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in heart_df.columns]).show()

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|   0|  0|      105|            0|        29|    53|              0|           0|       0|     50|    0|    0| 19|        1|    388|         0|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+



In [7]:
# Remove rows with at least one value as null

heart_df = heart_df.dropna()


In [8]:
heart_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in heart_df.columns]).show()

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|   0|  0|        0|            0|         0|     0|              0|           0|       0|      0|    0|    0|  0|        0|      0|         0|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+



In [19]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import StructType, StructField, FloatType

# Assemble the features into a single vector column
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=[
    "male", "age", "education", "currentSmoker", "cigsPerDay",
    "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes",
    "totChol", "sysBP", "diaBP", "BMI", "heartRate",
    "glucose"
], outputCol="features")

data_with_features = assembler.transform(heart_df).select("TenYearCHD", "features")

# Split the data into training and test sets 70% training data and 30% testing data
train, test = data_with_features.randomSplit([0.7, 0.3], seed=57)
# Create a logistic regression model
lr = LogisticRegression(maxIter=10, regParam=0.01, featuresCol="features", labelCol="TenYearCHD")
# Train to get the mod
lr_model = lr.fit(train)


In [20]:
# Print coefficients
print("Coefficients: " + str(lr_model.coefficients))

# Make predictions on the test set
predictions = lr_model.transform(test)

# Show predictions
predictions.select("features", "prediction", "probability").show()

Coefficients: [0.5728581837020535,0.05735994205785407,-0.020296087562109176,0.07579602918356385,0.012702171657120023,0.3450321084063676,0.5434492327558711,0.24254325051377393,0.2482727165880535,0.0009956737523568449,0.014509428371414959,2.6048904178987696e-05,0.000821202641470393,-0.0022055738259466164,0.007571027389351979]
+--------------------+----------+--------------------+
|            features|prediction|         probability|
+--------------------+----------+--------------------+
|(15,[1,2,9,10,11,...|       0.0|[0.97955921831705...|
|(15,[1,2,9,10,11,...|       0.0|[0.97353675648838...|
|(15,[1,2,9,10,11,...|       0.0|[0.97344272039882...|
|(15,[1,2,9,10,11,...|       0.0|[0.97758242748823...|
|(15,[1,2,9,10,11,...|       0.0|[0.96966771953813...|
|(15,[1,2,9,10,11,...|       0.0|[0.96757970201429...|
|(15,[1,2,9,10,11,...|       0.0|[0.97907527380872...|
|(15,[1,2,9,10,11,...|       0.0|[0.97829220533120...|
|(15,[1,2,9,10,11,...|       0.0|[0.94994649877908...|
|(15,[1,2,9,10